In [1]:
from funciones import clean_algabo, clean_furey, concat, clean_canal

In [1]:
from funciones import clean_algabo, clean_furey, concat, clean_canal, scrape_drimel


def clean_file(canal,filename, company):
    if company == "Algabo":
        clean_algabo(filename)
        clean_canal(canal,'canal',mpn_value=company)
        concat('./procesadosCanal/algabo.xlsx','./procesados/algabo.xlsx','./listos/algabo.xlsx')
    elif company == "Furey":
        clean_furey(filename)
        clean_canal(canal,'canal',mpn_value=company)
        concat('./procesadosCanal/furey.xlsx','./procesados/furey.xlsx','./listos/furey.xlsx')
    elif company == 'Drimel':
        scrape_drimel(filename)
        clean_canal(canal,'drimel',mpn_value=company)
        concat('./procesadosCanal/drimel.xlsx','./procesados/drimel.xlsx','./listos/drimel.xlsx')
    else:
        raise ValueError("Company not recognized")


In [2]:
clean_file('./sinProcesar/canal.csv',
           ['https://drimel.com.ar/?product_cat=bebes',
            'https://drimel.com.ar/?product_cat=cuidado-del-bebe'],
            'Drimel')

Archivo guardado con exito
Archivo limpio guardado como ./procesadosCanal/drimel.xlsx


/Users/franciscofurey/00DataScience/Canal/actualizadorPrecios/funciones.py:23: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Precio'] = df['Precio'].str.replace('\..*', '').str.replace(',', '')


ValueError: cannot set a row with mismatched columns

In [ ]:
#clean_file('./sinProcesar/canal.csv','./sinProcesar/furey.xlsx', 'Furey')

/Users/franciscofurey/00DataScience/Canal/actualizadorPrecios/funciones.py:200: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Código de barras'] = df['Código de barras'].str.replace('.', '').str.replace('+', '').str.replace('E', '').str.replace('-', '')


Archivo limpio guardado como ./procesados/furey.xlsx
Archivo limpio guardado como ./procesadosCanal/canal.xlsx


/Users/franciscofurey/00DataScience/Canal/actualizadorPrecios/funciones.py:18: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Precio'] = df['Precio'].str.replace('\..*', '').str.replace(',', '')


FileNotFoundError: [Errno 2] No such file or directory: './procesadosCanal/furey.xlsx'

In [11]:
clean_file('./sinProcesar/canal.csv','./sinProcesar/furey.xlsx', 'Furey')

/Users/franciscofurey/00DataScience/Canal/funciones.py:206: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  


Archivo limpio guardado como ./sinProcesar/../procesados/furey.xlsx


/Users/franciscofurey/00DataScience/Canal/funciones.py:18: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Precio'] = df['Precio'].str.replace('\..*', '').str.replace(',', '')


Archivo limpio guardado como ./sinProcesar/procesadosCanal/canal.xlsx


In [1]:
from funciones import clean_canal
clean_canal('./sinProcesar/canal.csv','furey',mpn_value='Furey')

/Users/franciscofurey/00DataScience/Canal/actualizadorPrecios/funciones.py:18: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Precio'] = df['Precio'].str.replace('\..*', '').str.replace(',', '')


Archivo limpio guardado como ./procesadosCanal/furey.xlsx


In [1]:
from funciones import concat
concat('./sinProcesar/procesadosCanal/canalFurey.xlsx','./procesados/furey.xlsx','./listos/fureyCanal.xlsx')

Archivo CONCATENADO guardado como ./listos/fureyCanal.xlsx


In [3]:
def concat(archivo1, archivo2, archivo_resultado):
    import os
    import pandas as pd
    import openpyxl
    from fuzzywuzzy import fuzz
    import requests
    import pandas as pd
    from bs4 import BeautifulSoup
    #import credentials
    import re
    # Cargar los dos archivos de Excel
    df1 = pd.read_excel(archivo1)
    df2 = pd.read_excel(archivo2)
    df1['Código de barras'] = df1['Código de barras'].astype(str)
    #
    df2['Código de barras'] = df2['Código de barras'].astype(str)

    df_result = pd.DataFrame(columns=['Identificador de URL','canal_Nombre','canal_Precio','canal_SKU','canal_Código de barras',
                                      'df2_SKU','df2_Nombre','df2_Código de barras','df2_Costo','similarity'])


    for i, row1 in df1.iterrows():
        canal_Nombre = row1['Nombre']
        canal_Precio = row1['Precio']
        canal_SKU = row1['SKU']
        canal_Codigo = row1['Código de barras']
        canal_Identificador = row1['Identificador de URL']
        found = False

        for j, row2 in df2.iterrows():
            df2_Nombre = row2['Nombre']
            df2_Costo = row2['Costo']
            df2_SKU = row2['SKU']
            df2_Codigo = row2['Código de barras']

            similarity = fuzz.token_set_ratio(canal_SKU, df2_SKU)

            if similarity >= 99:
                df_result.loc[len(df_result)] = [canal_Identificador, canal_Nombre, canal_Precio, canal_SKU, canal_Codigo,
                                                  df2_SKU, df2_Nombre, df2_Codigo, df2_Costo, similarity]
                if 'Marca' not in df_result.columns:
                    df_result['Marca'] = ''
                found = True
                    # Verificar si la columna "Marca" existe en df2
                if "Marca" in df2.columns:
                    # Si la columna existe, agregarla al dataframe df_result
                    df_result.loc[len(df_result) - 1, "Marca"] = row2["Marca"]
                else:
                    # Si la columna no existe, no hacer nada y continuar con la ejecución del código
                    pass

        if not found:
            df_result.loc[len(df_result)] = [canal_Identificador, canal_Nombre, canal_Precio, canal_SKU, canal_Codigo, 
                                             None, None, None, None, None]
            # Verificar si la columna "Marca" existe en df2
            if "Marca" in df2.columns:
                # Si la columna existe, agregarla al dataframe df_result
                df_result.loc[len(df_result) - 1, "Marca"] = None
            else:
                # Si la columna no existe, no hacer nada y continuar con la ejecución del código
                pass

    for i, row2 in df2.iterrows():
        df2_Nombre = row2['Nombre']
        df2_Costo = row2['Costo']
        df2_SKU = row2['SKU']
        df2_Codigo = row2['Código de barras']
        found = False

        for j, row_result in df_result.iterrows():
            if row_result['df2_SKU'] == df2_SKU:
                found = True
                break
        
        if not found:
            df_result.loc[len(df_result)] = [None, None, None, None, None, 
                                             df2_SKU, df2_Nombre, df2_Codigo, df2_Costo, None]
            
    df_result.to_excel(archivo_resultado, index= False, float_format = '%.15g')
    print("Archivo CONCATENADO guardado como",archivo_resultado)

In [5]:

concat('./procesadosCanal/drimel.xlsx','./procesados/drimel.xlsx','./listos/drimel.xlsx')

ValueError: cannot set a row with mismatched columns